In [1]:
import numpy as np
import matplotlib.pyplot as plt
import cv2

In [2]:
apple = cv2.imread('Images/apple.jpg')
orange = cv2. imread('Images/orange.jpg')
print(apple.shape)
print(orange.shape)

(512, 512, 3)
(512, 512, 3)


In [3]:
cv2.imshow('apple',apple)
cv2.imshow('orange',orange)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [5]:
# join half apple with half orange
apple_orange = np.hstack((apple[:,:256],orange[:,256:]))
cv2.imshow('apple_orange',apple_orange)
cv2.waitKey(0)
cv2.destroyAllWindows()
# we can see line in image
# we don't want that

In [6]:
# steps :
# 1. Load the two images of apple and orange.
# 2. Find the Gaussian Pyramids for apple and orange(in this particular example, number of levels is 6).
# 3. From Gaussian Pyramids, find their Laplacian Pyramids.
# 4. Now join the left half of apple and right half of orange in each levels of Laplacian Pyramids.
# 5. Finally from this joint image pyramids, reconstruct the original image.

In [7]:
# first step already done
# 2.
apple_copy = apple.copy()
gp_apple = [apple_copy]

for i in range(6):
    apple_copy = cv2.pyrDown(apple_copy)
    gp_apple.append(apple_copy)
    

orange_copy = orange.copy()
gp_orange = [orange_copy]

for i in range(6):
    orange_copy = cv2.pyrDown(orange_copy)
    gp_orange.append(orange_copy)

In [9]:
# 3.
apple_copy = gp_apple[5]
lp_apple = [apple_copy]
for i in range(5,0,-1):
    gaussian_expanded = cv2.pyrUp(gp_apple[i])
    laplacian = cv2.subtract(gp_apple[i-1],gaussian_expanded)
    lp_apple.append(laplacian)
    
orange_copy = gp_orange[5]
lp_orange = [orange_copy]
for i in range(5,0,-1):
    gaussian_expanded = cv2.pyrUp(gp_orange[i])
    laplacian = cv2.subtract(gp_orange[i-1],gaussian_expanded)
    lp_orange.append(laplacian)

In [10]:
# 4.
apple_orange_pyramid = []
n = 0
for apple_lap,orange_lap in zip(lp_apple,lp_orange):
    n += 1
    cols,rows,ch = apple_lap.shape
    laplacian = np.hstack((apple_lap[:,0:int(cols/2)],orange_lap[:,int(cols/2):]))
    apple_orange_pyramid.append(laplacian)
    

apple_orange_reconstruct = apple_orange_pyramid[0]
for i in range(1,6):
    apple_orange_reconstruct = cv2.pyrUp(apple_orange_reconstruct)
    apple_orange_reconstruct = cv2.add(apple_orange_pyramid[i],apple_orange_reconstruct)
    
    
cv2.imshow('apple_orange_reconstruct',apple_orange_reconstruct)
cv2.waitKey(0)
cv2.destroyAllWindows()